<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Stefano Schiappacasse

### **Link de repositorio de GitHub:** `https://github.com/stefanoschiappacasse/RepoMDS7202`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

# Importamos librerias utiles 😸

In [1]:
%%capture
# !pip install praw
# !pip install line_profiler
# !pip install memory_profiler
# !pip install numba
%load_ext line_profiler
%load_ext memory_profiler

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image
import plotly.graph_objects as go

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [61]:
def praw_reddit(nombre_subreddit="chile", n_hot=100000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [48]:
votes, post, url = praw_reddit()

In [49]:
#Aplicamos el perfilamiento por linea para estudiar los tiempos de ejecución
%lprun -f praw_reddit praw_reddit()

Podemos observar que aproximadamente 99.5% del tiempo de la función se va en obtener los primeros `n` post de la categoría `hot`. Esto tiene sentido ya que es el momento donde se realiza la consulta con el servidor externo y se traen los datos. Por lo tanto, si se quisiera realizar una mejora a esta función, ahí es donde habría que ponerle foco. A continuación veremos cómo se comporta la memoria en esta función.

In [50]:
%%file praw_reddit_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting praw_reddit_module.py


In [51]:
#Aplicamos el perfilamiento por linea para estudiar la memoria utilizada
from praw_reddit_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

Los resultados muestran que la función parte utilizando 201.4 Mb (este valor puede variar en una nueva ejecución) y luego la línea más incrementa el uso de memoria es la misma que consume más tiempo, la línea que obtiene los `top_n` post de la categoría `hot` (línea 16). Tiene sentido que sea ésta la línea que incrementa el uso de memoria ya que estoy guardando en una variable los `top_n` posts. Por lo tanto, a modo de conclusión, si quisieramos mejorar el desempeño de nuestra función, sería la línea 16 a la cual deberíamos ponerle mayor esfuerzo.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

1) La memoria caché es un espacio de memoria especial utilizado para guardar información que será utilizada en un futuro. Se puede almacenar tanto en la memoria ram, disco duro o almacenada de manera remota, siempre muy cerca de la CPU. La idea por detrás es no tener que volver a calcular o generar la información en un eventual paso, sino que leerla directamente del caché. La gracia es que el procesador tiene acceso directo a ella de manera casi instantánea y esto permiter mejorar el rendimiento y velocidad de la CPU en la ejecución de las tareas cotidianas. Las siglas LRU hacen referencia a un algoritmo de caché `Least Recently Used`. Un "algoritmo de caché" es un programa que optimiza la gestión de la información en la memoria caché del computador. El algoritmo LRU va descartando primero los elementos que menos se utilizan en la caché, llevando un seguimiento de lo que se usa y lo que no se usa. Para esto debe llevar la cuenta de la "antigüedad" de cada elemento del caché y buscar el menos usado.

2) Los costos tienen que ver con el aumento del consumo de la memoria. En el caso que esta memoria para el caché esté alojada en el disco, puede ralentizar el acceso a ésta y disminuir el rendimiendo. 

3) La consecuencia sería que tendríamos los datos obtenidos de reddit almacenados en espacio de memoria para poder acceder a ellos cuando los necesitemos de manera rápida. Esto permite hacer más rápido el acceso de acceder a ellos, y al mismo tiempo estaremos ocupando más espacio para almacenar estos datos. Sin embargo, si al ejecutar la función nuestra finalidad es traernos los post actualizados, entonces generar un caché para estos datos no sería una buena decisión, ya que tendría los mismos post almacenados cada vez que ejecuto la función.

**Respuesta de lru_cache sobre praw_reddit**

Primero mediremos el tiempo sin el caché.

In [62]:
#tiempo total de ejecución.
# tiempo inicial
t0 = time.time()

praw_reddit()

# tiempo final
t1 = time.time()

# el tiempo transcurrido es simplemente el delta entre t1 y t0
print("Tiempo transcurrido en segundos", t1 - t0)

Tiempo transcurrido en segundos 14.459930419921875


In [63]:
#tiempo promedio de ejeción por 
%timeit praw_reddit()

15.2 s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Podemos observar que el tiempo total de ejecución es de aproximadamente 15.6 segundos y el promedio por loop de 14.8 segundos (estos resultados pueden variar al ejecutar nuevamente las celdas). Si agregamos el caché, podemos ver los siguientes resultados.

In [64]:
@lru_cache
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [65]:
#tiempo total de ejecución.
# tiempo inicial
t0 = time.time()

praw_reddit()

# tiempo final
t1 = time.time()

# el tiempo transcurrido es simplemente el delta entre t1 y t0
print("Tiempo transcurrido en segundos", t1 - t0)

Tiempo transcurrido en segundos 15.719220399856567


In [66]:
#tiempo promedio de ejeción por 
%timeit praw_reddit()

71.3 ns ± 4.29 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Los resultados revelan que tiempo de ejecución de la función propiamente tal no varía mucho, aumentando en 1.4 segundos aproximadamente. Sin embargo, cuando observamos los tiempos de ejecución por loop, existe un aumento de rendimiento del orden de 10e-9, pasando de 15.2 segundos en promedio, a 71.3 nano segundos en promedio. También observar que se realizaron 10 millones más de loops por cada `run` de la función que está con caché, a diferencia de la función sin caché en donde sólo se realizó un loop por cada `run`. Estos resultados muestran que el utilizar memoria caché sólo tiene sentido cuando la función se ejecutará más de una vez.

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [69]:
#Función que calcula la mean posterior y el estándar error sin utilizar numpy.
def muSigmaSinNumpy(votos):
    import numpy as np
    mu = []
    sigma =[]
    for voto in votos:
        a = np.int64(1 + voto[0])
        b = np.int64(1 + voto[1])
        sigma.append(1.65 * (np.sqrt((a * b)/(((a + b)**2) * (a + b + 1)))))
        mu.append(a /(a + b))
    return mu, sigma

In [104]:
#Función que calcula la mean posterior y el estándar error utilizando numpy.
def muSigmaConNumpy(votos):
    import numpy as np
    a = np.int64(votos[:, 0] + 1)
    b = np.int64(votos[:, 1] + 1)
    sigma = 1.65 * (np.sqrt((a * b)/(((a + b)**2) * (a + b + 1))))
    mu = a /(a + b)
    return mu, sigma

## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

In [106]:
#creación de gráficos para las funciones con y sin numpy.
def creacionGraficoSinNumba(arregloVotos):
    #arreglos que contendrán los resultados
    batchesSinNumpy = []
    tiemposSinNumpy = []
    batchesConNumpy = []
    tiemposConNumpy = []
    #iteración por todos los batches de posts.
    for batch, votos in arregloVotos.items():
        #cálculo de los tiempos de ejecución para la función sin numpy.
        t0SinNumpy = time.time_ns()
        muSinNumpy, sigmaSinNumpy = muSigmaSinNumpy(votos)
        t1SinNumpy = time.time_ns()
        tiempoSinNumpy = t1SinNumpy-t0SinNumpy
        batchesSinNumpy.append(f'batch: {votos.shape[0]}')
        tiemposSinNumpy.append(tiempoSinNumpy)
        
        #cálculo de los tiempos de ejecución para la función con numpy.
        t0ConNumpy = time.time_ns()
        muConNumpy, sigmaConNumpy = muSigmaConNumpy(votos)
        t1ConNumpy = time.time_ns()
        tiempoConNumpy = t1ConNumpy-t0ConNumpy
        batchesConNumpy.append(f'batch: {votos.shape[0]}')
        tiemposConNumpy.append(tiempoConNumpy)
    
    #consolidación de resultados.
    resultadosSinNumpy = {'batch': batchesSinNumpy, 'tiempo': tiemposSinNumpy}
    resultadosConNumpy = {'batch': batchesConNumpy, 'tiempo': tiemposConNumpy}
    return pd.DataFrame(data = resultadosSinNumpy), pd.DataFrame(data = resultadosConNumpy) 

In [108]:
#creación de dataframes con los resultados con y sin numpy
dfSinNumpy, dfConNumpy = creacionGraficoSinNumba(votes)

batches = dfSinNumpy['batch']

fig = go.Figure()

fig.add_trace(go.Scatter(x=batches, y=dfSinNumpy['tiempo'], name='Resultados SinNumpy',
                         line=dict(color='firebrick', width=4, dash='dot')))
fig.add_trace(go.Scatter(x=batches, y=dfConNumpy['tiempo'], name = 'Resultados ConNumpy',
                         line=dict(color='royalblue', width=4, dash='dot')))

fig.update_layout(title='Resultados funciones con y sin Numpy',
                   xaxis_title='Tamaño batches',
                   yaxis_title='Tiempos en nano segundos')

fig.show()

Los resultados reflejan que para la función sin numpy, las cantidades de datos influyen considerablemente en el tiempo de ejecución de ésta. No así para la función con numpy, en donde se ven unos aumentos en los tiempos de ejecución de la función en algunos batches (50, 350, 400, 550). Notar que curiosamente existen algunos tamaños de batches que aumentan los tiempos de ejecución, a pesar de tener menos datos que otros batches, como por ejemplo para el caso de los resultados con numpy, el batch 550 posee menos datos que el batch de 800, pero posee un tiempo de ejecución mayor. Esto se debe a cómo funciona la función por debajo en donde va guardando ciertos espacios de memoria para alojar los datos, y cuando éstos se copan, se debe pedir más espacio y copiar todos los datos al nuevo lugar, haciendo que aumente el tiempo de ejecución.

Ahora aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

In [121]:
from numba import jit
import numpy as np

#función sin numpy aplicando Numba
@jit(nopython=True)
def muSigmaSinNumpyConNumba(votos):
    mu = []
    sigma =[]
    for voto in votos:
        a = np.int64(1 + voto[0])
        b = np.int64(1 + voto[1])
        sigma.append(1.65 * (np.sqrt((a * b)/(((a + b)**2) * (a + b + 1)))))
        mu.append(a /(a + b))
    return mu, sigma

In [139]:
from numba import jit
import numpy as np

#función con numpy aplicando Numba
@jit(nopython=True)
def muSigmaConNumpyConNumba(votos):
    a = (votos[:, 0] + 1).astype(np.uint64)
    b = (votos[:, 1]+ 1).astype(np.uint64)
    sigma = 1.65 * (np.sqrt((a * b)/(((a + b)**2) * (a + b + 1))))
    mu = a /(a + b)
    return mu, sigma

Como menciona el enunciado, primero se ejecutan las funciones para que se compilen y luego se obtendrán los tiempos.

In [123]:
muSigmaSinNumpyConNumba(votes[0])

([0.7662337662337663,
  0.9285714285714286,
  0.9881481481481481,
  0.9673202614379085,
  0.9125,
  0.965034965034965,
  0.9864864864864865,
  0.9101123595505618,
  0.9230769230769231,
  0.98,
  0.9583333333333334,
  0.8541666666666666,
  0.8955223880597015,
  0.8769230769230769,
  0.8740157480314961,
  0.8888888888888888,
  0.9957627118644068,
  0.96875,
  0.7884615384615384,
  0.8932038834951457,
  0.8695652173913043,
  0.9175627240143369,
  0.6666666666666666,
  0.7407407407407407,
  0.8409090909090909,
  0.9089635854341737,
  0.6857142857142857,
  0.8571428571428571,
  0.9310344827586207,
  0.9567901234567902,
  0.9523809523809523,
  0.956140350877193,
  0.7865168539325843,
  0.7586206896551724,
  0.9,
  0.8823529411764706,
  0.6785714285714286,
  0.8333333333333334,
  0.7586206896551724,
  0.7,
  0.9855595667870036,
  0.832,
  0.9396551724137931,
  0.9785575048732943,
  0.8611111111111112,
  0.95,
  0.9041095890410958,
  0.896551724137931,
  0.9851851851851852,
  0.893805309734513

In [140]:
muSigmaConNumpyConNumba(votes[0])

(array([0.76623377, 0.92857143, 0.98814815, 0.96732026, 0.9125    ,
        0.96503497, 0.98648649, 0.91011236, 0.92307692, 0.98      ,
        0.95833333, 0.85416667, 0.89552239, 0.87692308, 0.87401575,
        0.88888889, 0.99576271, 0.96875   , 0.78846154, 0.89320388,
        0.86956522, 0.91756272, 0.66666667, 0.74074074, 0.84090909,
        0.90896359, 0.68571429, 0.85714286, 0.93103448, 0.95679012,
        0.95238095, 0.95614035, 0.78651685, 0.75862069, 0.9       ,
        0.88235294, 0.67857143, 0.83333333, 0.75862069, 0.7       ,
        0.98555957, 0.832     , 0.93965517, 0.9785575 , 0.86111111,
        0.95      , 0.90410959, 0.89655172, 0.98518519, 0.89380531]),
 array([0.07906935, 0.0504311 , 0.00686775, 0.01367822, 0.03674443,
        0.02525755, 0.01275737, 0.04974623, 0.03250167, 0.03234647,
        0.04710192, 0.05912863, 0.06120393, 0.04736062, 0.04839458,
        0.05435828, 0.00696196, 0.04997558, 0.09256162, 0.04997126,
        0.11342952, 0.02711968, 0.08436615, 0.

Ahora compararemos los tiempos de ejecución sobre el batch más grande del dataset de votos. Partiremos comparando las funciones sin numpy.

In [141]:
t0 = time.time_ns()
muSinNumpy, sigmaSinNumpy = muSigmaSinNumpy(votes[len(votes) - 1])
t1 = time.time_ns()
print(f'El tiempo de ejecución de la función sin numpy y sin numba es: {t1-t0} nanosegundos')

t0 = time.time_ns()
muConNumpy, sigmaConNumpy = muSigmaSinNumpyConNumba(votes[len(votes) - 1])
t1 = time.time_ns()
print(f'El tiempo de ejecución de la función sin numpy y sin numba es: {t1-t0} nanosegundos')

El tiempo de ejecución de la función sin numpy y sin numba es: 4984000 nanosegundos
El tiempo de ejecución de la función sin numpy y sin numba es: 0 nanosegundos


Increíble la diferencia entre ambas funciones. La función compilada se demora 0 nanosegundos versus los casi 5 millones de nanosegundos que se demora la función sin numba (este valor puede variar en otras ejecuciones de la celda). Esto demuestra un aumento de rendimiendo del orden de los millones. Ahora verificaremos los resultados de las funciones con numpy.

In [142]:
t0 = time.time_ns()
muSinNumpy, sigmaSinNumpy = muSigmaConNumpy(votes[len(votes) - 1])
t1 = time.time_ns()
print(f'El tiempo de ejecución de la función con numpy y sin numba es: {t1-t0} nanosegundos')

t0 = time.time_ns()
muConNumpy, sigmaConNumpy = muSigmaConNumpyConNumba(votes[len(votes) - 1])
t1 = time.time_ns()
print(f'El tiempo de ejecución de la función con numpy y sin numba es: {t1-t0} nanosegundos')

El tiempo de ejecución de la función con numpy y sin numba es: 1000000 nanosegundos
El tiempo de ejecución de la función con numpy y sin numba es: 0 nanosegundos


Nuevamente podemos observar cómo la función compilada sobresale respecto a la sin compilar. La mejora del rendimiento está cerca del orden del millón de nanosegundos.

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

In [143]:
def creacionGraficoConNumba(arregloVotos):
    batchesSinNumpy = []
    tiemposSinNumpy = []
    batchesConNumpy = []
    tiemposConNumpy = []
    for batch, votos in arregloVotos.items():
        t0SinNumpy = time.time_ns()
        muSinNumpy, sigmaSinNumpy = muSigmaSinNumpyConNumba(votos)
        t1SinNumpy = time.time_ns()
        tiempoSinNumpy = t1SinNumpy-t0SinNumpy
        batchesSinNumpy.append(f'batch: {votos.shape[0]}')
        tiemposSinNumpy.append(tiempoSinNumpy)
        
        t0ConNumpy = time.time_ns()
        muConNumpy, sigmaConNumpy = muSigmaConNumpyConNumba(votos)
        t1ConNumpy = time.time_ns()
        tiempoConNumpy = t1ConNumpy-t0ConNumpy
        batchesConNumpy.append(f'batch: {votos.shape[0]}')
        tiemposConNumpy.append(tiempoConNumpy)
    resultadosSinNumpy = {'batch': batchesSinNumpy, 'tiempo': tiemposSinNumpy}
    resultadosConNumpy = {'batch': batchesConNumpy, 'tiempo': tiemposConNumpy}
    return pd.DataFrame(data = resultadosSinNumpy), pd.DataFrame(data = resultadosConNumpy) 

In [144]:
dfSinNumpyConNumba, dfConNumpyConNumba = creacionGraficoConNumba(votes)

In [145]:
import plotly.graph_objects as go

# Add data
batches = dfSinNumpy['batch']

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=batches, y=dfSinNumpy['tiempo'], name='Resultados SinNumpy SinNumba',
                         line=dict(color='firebrick', width=4, dash='dot')))
fig.add_trace(go.Scatter(x=batches, y=dfConNumpy['tiempo'], name = 'Resultados ConNumpy SinNumba',
                         line=dict(color='royalblue', width=4, dash='dot')))
fig.add_trace(go.Scatter(x=batches, y=dfSinNumpyConNumba['tiempo'], name = 'Resultados SinNumpy ConNumba',
                         line=dict(color='firebrick', width=4, dash='dash')))
fig.add_trace(go.Scatter(x=batches, y=dfConNumpyConNumba['tiempo'], name = 'Resultados ConNumpy ConNumba',
                         line = dict(color='royalblue', width=4, dash='dash')))
# Edit the layout
fig.update_layout(title='Resultados distintas funciones',
                   xaxis_title='Tamaño batches',
                   yaxis_title='Tiempos en nano segundos')


fig.show()

Los resultados muestran que las funciones compilada en general se ejecutan considerablemente más rápido que las sin compilar. Debido a que numpy por debajo igual tienes métodos pre compilados, la mejora no se aprecia tanto cuando lo comparamos con las funciones en las que se implementó numba. Sin embargo, si vemos la función en donde no se utiliza ni numba ni numpy, y la comparamos con la misma función pero utilizando numba, se puede observar que en el batch de 800 registros, esta última, supera en el orden de los 5 millones de nanosegundos a la función que no utiliza numba, lo cual representa una mejoría en el rendimiendo realmente considerable.

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [150]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = muSigmaConNumpy(votos)
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df1 = pd.DataFrame(data=vote_post)
ordered_post = df1.Post

r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df2 = pd.DataFrame(data=data_dic)
fig = px.scatter(df2, x="mean", y="post", error_x="std_err")
fig.show()

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [154]:
pd.concat([df1, df2], axis = 1)

,Votos (+),Votos (-),Post,url,mean,std_err,post
0,761,0,"Niños en la Vega Central de Santiago, septiemb...",https://i.redd.it/xx9dj27wfe281.jpg,0.998689,0.002160,"Niños en la Vega Central de Santiago, septiemb..."
1,626,0,Mi viejo nos mando estas fotos de un visitante...,https://www.reddit.com/gallery/r4fe4i,0.998408,0.002623,Mi viejo nos mando estas fotos de un visitante...
2,312,0,Me encontré un tramoya de 31 minutos en la feria,https://i.redd.it/urjnpbvv3y281.jpg,0.996815,0.005238,Me encontré un tramoya de 31 minutos en la feria
3,277,0,Andando en bici me encontré a esta linda parej...,https://www.reddit.com/gallery/r50htt,0.996416,0.005893,Andando en bici me encontré a esta linda parej...
4,239,0,Eclipse Solar desde la base chilena Glaciar Un...,https://i.redd.it/538k965hqj381.jpg,0.995851,0.006818,Eclipse Solar desde la base chilena Glaciar Un...
5,234,0,Temuco a las 1:30 AM.,https://v.redd.it/ombhs2no22481,0.995763,0.006962,Temuco a las 1:30 AM.
6,226,0,Chanchoman y La Mosca de Cachureos detrás de c...,https://i.redd.it/wvoixdu2am281.jpg,0.995614,0.007205,Chanchoman y La Mosca de Cachureos detrás de c...
7,1099,11,Compromisos son compromisos,https://i.redd.it/cqgsc15az3181.jpg,0.989209,0.005110,Compromisos son compromisos
8,798,8,Abeja atrapada en telaraña siendo ayudada por ...,https://v.redd.it/ui76swk5pf281,0.988861,0.006088,Abeja atrapada en telaraña siendo ayudada por ...
9,787,8,Un día cualquiera en Iquique,https://v.redd.it/xt7ifxo33c181,0.988708,0.006172,Un día cualquiera en Iquique


**Respuesta:**

Al observar la visualización en conjunto con los dataframes generados, es posible observar que los post que poseen una mayor probabilidad de ser catalogados como buenos (según la mean posterior) son aquellos que poseen gran cantidad de votos positivos y ningún voto negativo. Se les da mayor probabilidad a éstos post, por sobre los que tienen más votos positivos pero también algunos negativos. Un ejemplo de ésto se observa en las filas 6 y 7 en donde la primera posee 226 votos positivos y 0 negativos, y la segunda cuatruplica los votos positivos de la primera, pero posee 11 votos negativos lo que hace que obtenga una menor mean posterior.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>